In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import accuracy_score

C:\Users\krizm\AppData\Local\Temp\ipykernel_10316\948096407.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the datasets
df_2021 = pd.read_csv('2021data.csv')
df_2022 = pd.read_csv('2022data.csv')
df_2023 = pd.read_csv('2023data.csv')

# Combine the datasets
data = pd.concat([df_2021, df_2022, df_2023])

# Save the combined dataset
data.to_csv('combined_dataset.csv', index=False)

In [3]:
data

,Week,Team,Day,Date,Time,Link,Result,OT,Rec,H/A,...,Off_RushYd,Off_TO,Def_1stD,Def_TotYd,Def_PassYd,Def_RushYD,Def_TO,Exp_Offense,Exp_Defense,Exp_SpecTms
0,1,49ers,Sun,12-Sep,1:00PM ET,boxscore,W,NaN,Jan-00,@,...,131.0,2.0,31.0,430.0,314.0,116.0,1.0,15.48,-5.16,-1.73
1,2,49ers,Sun,19-Sep,1:00PM ET,boxscore,W,NaN,Feb-00,@,...,117.0,NaN,18.0,328.0,177.0,151.0,NaN,7.86,-5.30,2.68
2,3,49ers,Sun,26-Sep,8:20PM ET,boxscore,L,NaN,1-Feb,NaN,...,67.0,2.0,21.0,353.0,253.0,100.0,NaN,12.19,-15.14,0.95
3,4,49ers,Sun,3-Oct,4:05PM ET,boxscore,L,NaN,2-Feb,NaN,...,143.0,2.0,14.0,234.0,129.0,105.0,NaN,8.61,-2.20,-14.31
4,5,49ers,Sun,10-Oct,4:25PM ET,boxscore,L,NaN,3-Feb,@,...,152.0,1.0,20.0,304.0,210.0,94.0,1.0,-6.53,-1.74,-0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,14,vikings,Sun,10-Dec,4:05PM ET,boxscore,W,NaN,6-Jul,@,...,132.0,NaN,8.0,202.0,146.0,56.0,3.0,-10.05,20.46,-6.43
612,15,vikings,Sat,16-Dec,1:00PM ET,boxscore,L,OT,7-Jul,@,...,143.0,2.0,22.0,378.0,298.0,80.0,1.0,8.62,-9.29,-1.35
613,16,vikings,Sun,24-Dec,1:00PM ET,boxscore,L,NaN,8-Jul,NaN,...,17.0,4.0,28.0,389.0,246.0,143.0,1.0,11.43,-15.44,-0.61
614,17,vikings,Sun,31-Dec,8:20PM ET,boxscore,L,NaN,9-Jul,NaN,...,67.0,2.0,28.0,470.0,293.0,177.0,1.0,-14.27,-22.54,12.19


In [4]:
data['Result'] = data['Result'].apply(lambda x: 1 if x == 'W' else 0)  # Convert Win to 1 and anything else to 0

#1 if OT, 0 if no OT
data['OT'] = data['OT'].fillna(0).replace('OT', 1)

# 1 is home, 0 is away
data['H/A'] = data['H/A'].fillna(1).replace('@', 0)

data = data.drop(columns = ['Day', 'Date', 'Time', 'Link', 'Rec', 'Week'])

data = data.dropna()

data = data.reset_index()

data = data.drop(columns = ['index'])
data['Score_Tm'] = data['Score_Tm'].astype(float)
data = data[~(data['H/A'] == 'N')]
data['H/A'] = data['H/A'].astype(float)
data['OT'] = data['OT'].astype(float)
data['Result'] = data['Result'].astype(float)
column = data.pop('Opp')
data.insert(1, 'Opp', column)

C:\Users\krizm\AppData\Local\Temp\ipykernel_10316\3462032.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['OT'] = data['OT'].fillna(0).replace('OT', 1)


In [5]:
data.head()

,Team,Opp,Result,OT,H/A,Score_Tm,Score_Opp,Off_1stD,Off_TotYd,Off_PassYd,Off_RushYd,Off_TO,Def_1stD,Def_TotYd,Def_PassYd,Def_RushYD,Def_TO,Exp_Offense,Exp_Defense,Exp_SpecTms
0,49ers,Detroit Lions,1.0,0.0,0.0,41.0,33.0,21.0,442.0,311.0,131.0,2.0,31.0,430.0,314.0,116.0,1.0,15.48,-5.16,-1.73
1,49ers,Arizona Cardinals,0.0,0.0,0.0,10.0,17.0,19.0,338.0,186.0,152.0,1.0,20.0,304.0,210.0,94.0,1.0,-6.53,-1.74,-0.47
2,49ers,Indianapolis Colts,0.0,0.0,1.0,18.0,30.0,13.0,280.0,169.0,111.0,4.0,17.0,295.0,147.0,148.0,2.0,-15.30,-1.16,4.24
3,49ers,Minnesota Vikings,1.0,0.0,1.0,34.0,26.0,23.0,423.0,215.0,208.0,1.0,17.0,323.0,256.0,67.0,2.0,8.93,4.16,-6.10
4,49ers,Seattle Seahawks,0.0,0.0,0.0,23.0,30.0,17.0,365.0,294.0,71.0,3.0,21.0,327.0,181.0,146.0,3.0,-0.51,-2.27,-3.44


In [6]:
df_2021

,Week,Team,Day,Date,Time,Link,Result,OT,Rec,H/A,...,Off_RushYd,Off_TO,Def_1stD,Def_TotYd,Def_PassYd,Def_RushYD,Def_TO,Exp_Offense,Exp_Defense,Exp_SpecTms
0,1,49ers,Sun,12-Sep,1:00PM ET,boxscore,W,NaN,Jan-00,@,...,131.0,2.0,31.0,430.0,314.0,116.0,1.0,15.48,-5.16,-1.73
1,2,49ers,Sun,19-Sep,1:00PM ET,boxscore,W,NaN,Feb-00,@,...,117.0,NaN,18.0,328.0,177.0,151.0,NaN,7.86,-5.30,2.68
2,3,49ers,Sun,26-Sep,8:20PM ET,boxscore,L,NaN,1-Feb,NaN,...,67.0,2.0,21.0,353.0,253.0,100.0,NaN,12.19,-15.14,0.95
3,4,49ers,Sun,3-Oct,4:05PM ET,boxscore,L,NaN,2-Feb,NaN,...,143.0,2.0,14.0,234.0,129.0,105.0,NaN,8.61,-2.20,-14.31
4,5,49ers,Sun,10-Oct,4:25PM ET,boxscore,L,NaN,3-Feb,@,...,152.0,1.0,20.0,304.0,210.0,94.0,1.0,-6.53,-1.74,-0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,14,vikings,Thu,9-Dec,8:20PM ET,boxscore,W,NaN,7-Jun,NaN,...,242.0,2.0,22.0,375.0,269.0,106.0,1.0,8.68,-6.41,1.31
612,15,vikings,Mon,20-Dec,8:15PM ET,boxscore,W,NaN,7-Jul,@,...,132.0,1.0,24.0,370.0,255.0,115.0,3.0,-4.65,7.72,5.92
613,16,vikings,Sun,26-Dec,1:00PM ET,boxscore,L,NaN,8-Jul,NaN,...,66.0,1.0,22.0,356.0,197.0,159.0,3.0,-1.81,0.84,-7.80
614,17,vikings,Sun,2-Jan,8:20PM ET,boxscore,L,NaN,9-Jul,@,...,27.0,NaN,29.0,481.0,307.0,174.0,NaN,-13.56,-21.96,7.80


# THE GOOD STUFF

In [7]:
df_2021 = pd.read_csv('2021data.csv')
df_2022 = pd.read_csv('2022data.csv')
df_2023 = pd.read_csv('2023data.csv')

In [8]:
# Convert Win to 1 and anything else to 0
df_2021['Result'] = df_2021['Result'].apply(lambda x: 1 if x == 'W' else 0) 

# 1 if OT, 0 if no OT
df_2021['OT'] = df_2021['OT'].fillna(0).replace('OT', 1)

# 1 is home, 0 is away
df_2021['H/A'] = df_2021['H/A'].fillna(1).replace('@', 0)

df_2021 = df_2021.drop(columns=['Day', 'Date', 'Time', 'Link', 'Rec', 'Week'])
df_2021 = df_2021.dropna()
df_2021 = df_2021.reset_index()
df_2021 = df_2021.drop(columns=['index'])
df_2021['Score_Tm'] = df_2021['Score_Tm'].astype(float)
df_2021 = df_2021[~(df_2021['H/A'] == 'N')]
df_2021['H/A'] = df_2021['H/A'].astype(float)
df_2021['OT'] = df_2021['OT'].astype(float)
df_2021['Result'] = df_2021['Result'].astype(float)
df_2021.pop('Opp')

df_2021.head()

C:\Users\krizm\AppData\Local\Temp\ipykernel_10316\3988736548.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2021['OT'] = df_2021['OT'].fillna(0).replace('OT', 1)


,Team,Result,OT,H/A,Score_Tm,Score_Opp,Off_1stD,Off_TotYd,Off_PassYd,Off_RushYd,Off_TO,Def_1stD,Def_TotYd,Def_PassYd,Def_RushYD,Def_TO,Exp_Offense,Exp_Defense,Exp_SpecTms
0,49ers,1.0,0.0,0.0,41.0,33.0,21.0,442.0,311.0,131.0,2.0,31.0,430.0,314.0,116.0,1.0,15.48,-5.16,-1.73
1,49ers,0.0,0.0,0.0,10.0,17.0,19.0,338.0,186.0,152.0,1.0,20.0,304.0,210.0,94.0,1.0,-6.53,-1.74,-0.47
2,49ers,0.0,0.0,1.0,18.0,30.0,13.0,280.0,169.0,111.0,4.0,17.0,295.0,147.0,148.0,2.0,-15.30,-1.16,4.24
3,49ers,1.0,0.0,1.0,34.0,26.0,23.0,423.0,215.0,208.0,1.0,17.0,323.0,256.0,67.0,2.0,8.93,4.16,-6.10
4,49ers,0.0,0.0,0.0,23.0,30.0,17.0,365.0,294.0,71.0,3.0,21.0,327.0,181.0,146.0,3.0,-0.51,-2.27,-3.44


In [9]:
# Convert Win to 1 and anything else to 0
df_2022['Result'] = df_2022['Result'].apply(lambda x: 1 if x == 'W' else 0) 

# 1 if OT, 0 if no OT
df_2022['OT'] = df_2022['OT'].fillna(0).replace('OT', 1)

# 1 is home, 0 is away
df_2022['H/A'] = df_2022['H/A'].fillna(1).replace('@', 0)

df_2022 = df_2022.drop(columns=['Day', 'Date', 'Time', 'Link', 'Rec', 'Week'])
df_2022 = df_2022.dropna()
df_2022 = df_2022.reset_index()
df_2022 = df_2022.drop(columns=['index'])
df_2022['Score_Tm'] = df_2022['Score_Tm'].astype(float)
df_2022 = df_2022[~(df_2022['H/A'] == 'N')]
df_2022['H/A'] = df_2022['H/A'].astype(float)
df_2022['OT'] = df_2022['OT'].astype(float)
df_2022['Result'] = df_2022['Result'].astype(float)
df_2022.pop('Opp')

df_2022.head()

C:\Users\krizm\AppData\Local\Temp\ipykernel_10316\288872902.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2022['OT'] = df_2022['OT'].fillna(0).replace('OT', 1)


,Team,Result,OT,H/A,Score_Tm,Score_Opp,Off_1stD,Off_TotYd,Off_PassYd,Off_RushYd,Off_TO,Def_1stD,Def_TotYd,Def_PassYd,Def_RushYD,Def_TO,Exp_Offense,Exp_Defense,Exp_SpecTms
0,49ers,0.0,0.0,0.0,10.0,19.0,17.0,331.0,155.0,176.0,2.0,15.0,204.0,105.0,99.0,1.0,-11.26,-0.67,0.42
1,49ers,1.0,0.0,0.0,37.0,15.0,22.0,397.0,244.0,153.0,1.0,15.0,308.0,244.0,64.0,1.0,16.04,13.22,-7.82
2,49ers,0.0,0.0,1.0,23.0,44.0,25.0,444.0,343.0,101.0,3.0,24.0,529.0,417.0,112.0,2.0,3.59,-30.58,8.50
3,49ers,1.0,0.0,1.0,22.0,16.0,22.0,387.0,230.0,157.0,1.0,12.0,238.0,187.0,51.0,1.0,6.20,7.29,-5.38
4,49ers,1.0,0.0,1.0,33.0,17.0,24.0,351.0,230.0,121.0,1.0,14.0,308.0,275.0,33.0,4.0,1.78,13.36,1.00


In [10]:
# Convert Win to 1 and anything else to 0
df_2023['Result'] = df_2023['Result'].apply(lambda x: 1 if x == 'W' else 0) 

# 1 if OT, 0 if no OT
df_2023['OT'] = df_2023['OT'].fillna(0).replace('OT', 1)

# 1 is home, 0 is away
df_2023['H/A'] = df_2023['H/A'].fillna(1).replace('@', 0)

df_2023 = df_2023.drop(columns=['Day', 'Date', 'Time', 'Link', 'Rec', 'Week'])
df_2023 = df_2023.dropna()
df_2023 = df_2023.reset_index()
df_2023 = df_2023.drop(columns=['index'])
df_2023['Score_Tm'] = df_2023['Score_Tm'].astype(float)
df_2023 = df_2023[~(df_2023['H/A'] == 'N')]
df_2023['H/A'] = df_2023['H/A'].astype(float)
df_2023['OT'] = df_2023['OT'].astype(float)
df_2023['Result'] = df_2023['Result'].astype(float)
df_2023.pop('Opp')

df_2023.head()

C:\Users\krizm\AppData\Local\Temp\ipykernel_10316\3475630129.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2023['OT'] = df_2023['OT'].fillna(0).replace('OT', 1)


,Team,Result,OT,H/A,Score_Tm,Score_Opp,Off_1stD,Off_TotYd,Off_PassYd,Off_RushYd,Off_TO,Def_1stD,Def_TotYd,Def_PassYd,Def_RushYD,Def_TO,Exp_Offense,Exp_Defense,Exp_SpecTms
0,49ers,1.0,0.0,0.0,30.0,7.0,22.0,391.0,203.0,188.0,1.0,15.0,239.0,198.0,41.0,2.0,9.73,13.55,-0.78
1,49ers,1.0,0.0,1.0,42.0,10.0,25.0,421.0,251.0,170.0,1.0,8.0,197.0,140.0,57.0,4.0,12.84,23.12,-5.21
2,49ers,0.0,0.0,0.0,17.0,19.0,15.0,215.0,107.0,108.0,1.0,18.0,334.0,174.0,160.0,2.0,-6.05,11.32,-7.34
3,49ers,0.0,0.0,0.0,17.0,22.0,19.0,325.0,260.0,65.0,3.0,24.0,452.0,378.0,74.0,1.0,5.43,-16.30,6.07
4,49ers,0.0,0.0,1.0,17.0,31.0,24.0,460.0,347.0,113.0,3.0,29.0,400.0,266.0,134.0,1.0,9.25,-18.30,2.48


In [11]:
agg_2021 = df_2021.groupby(df_2021.columns[0]).mean().reset_index()

agg_2022 = df_2021.groupby(df_2022.columns[0]).mean().reset_index()

agg_2023 = df_2021.groupby(df_2023.columns[0]).mean().reset_index()

In [12]:
agg_2021.head()

,Team,Result,OT,H/A,Score_Tm,Score_Opp,Off_1stD,Off_TotYd,Off_PassYd,Off_RushYd,Off_TO,Def_1stD,Def_TotYd,Def_PassYd,Def_RushYD,Def_TO,Exp_Offense,Exp_Defense,Exp_SpecTms
0,49ers,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,133.636364,1.636364,19.272727,309.727273,219.090909,90.636364,1.454545,3.477273,0.251818,-1.626364
1,bears,0.500000,0.000000,0.625000,17.625000,18.500000,18.625000,331.750000,212.750000,119.000000,2.000000,18.500000,271.625000,154.375000,117.250000,1.875000,-2.573750,3.120000,-0.567500
2,bengals,0.600000,0.200000,0.300000,26.900000,20.400000,18.900000,331.700000,223.200000,108.500000,1.700000,19.200000,342.300000,250.800000,91.500000,2.100000,2.364000,2.279000,2.332000
3,bills,0.666667,0.000000,0.444444,28.666667,12.111111,22.666667,358.333333,225.111111,133.222222,1.777778,14.777778,249.111111,146.111111,103.000000,2.111111,8.024444,9.343333,-1.233333
4,broncos,0.571429,0.000000,0.571429,19.857143,16.428571,20.428571,346.285714,221.857143,124.428571,1.285714,18.142857,317.000000,211.428571,105.571429,1.428571,4.684286,1.810000,-1.164286


In [13]:
df_2022 = pd.read_csv('2022data.csv')
df_2023 = pd.read_csv('2023data.csv')
#df_2024

mask = df_2022['Week'].isin(['Wild Card', 'Division', 'Conf. Champ.', 'SuperBowl'])
df_2022 = df_2022[~mask]
df_2022 = df_2022.dropna(subset = ['Opp'])
df_2022 = df_2022[df_2022['Opp'] != 'Bye Week'].reset_index()

mask = df_2023['Week'].isin(['Wild Card', 'Division', 'Conf. Champ.', 'SuperBowl'])
df_2023 = df_2023[~mask]
df_2023 = df_2023.dropna(subset = ['Opp'])
df_2023 = df_2023[df_2023['Opp'] != 'Bye Week'].reset_index()

In [14]:
wl_2022 = df_2022[['Result']]
wl_2023 = df_2023[['Result']]

df_2022 = df_2022[['Week', 'Team', 'Opp']]
df_2023 = df_2023[['Week', 'Team', 'Opp']]
#df_2024

In [15]:
# Create a mapping dictionary to convert team names
team_mapping = {
    'Chicago Bears': 'bears',
    'Seattle Seahawks': 'seahawks',
    'Denver Broncos': 'broncos',
    'Los Angeles Rams': 'rams',
    'Carolina Panthers': 'panthers',
    'Atlanta Falcons': 'falcons',
    'Kansas City Chiefs': 'chiefs',
    'Los Angeles Chargers': 'chargers',
    'Arizona Cardinals': 'cardinals',
    'New Orleans Saints': 'saints',
    'Miami Dolphins': 'dolphins',
    'Tampa Bay Buccaneers': 'buccaneers',
    'Washington Commanders': 'commanders',
    'Las Vegas Raiders': 'raiders',
    'Dallas Cowboys': 'cowboys',
    'Philadelphia Eagles': 'eagles',
    'San Francisco 49ers': '49ers',
    'Green Bay Packers': 'packers',
    'Houston Texans': 'texans',
    'New York Giants': 'giants',
    'Minnesota Vikings': 'vikings',
    'New England Patriots': 'patriots',
    'Detroit Lions': 'lions',
    'New York Jets': 'jets',
    'Buffalo Bills': 'bills',
    'Pittsburgh Steelers': 'steelers',
    'Baltimore Ravens': 'ravens',
    'Cleveland Browns': 'browns',
    'Tennessee Titans': 'titans',
    'Cincinnati Bengals': 'bengals',
    'Indianapolis Colts': 'colts',
    'Jacksonville Jaguars': 'jaguars'
}

# Map team names to their abbreviated versions in the 'Opp' column
df_2022['Opp'] = df_2022['Opp'].map(team_mapping)
df_2023['Opp'] = df_2023['Opp'].map(team_mapping)

In [16]:
df_2022_team = df_2022[['Week', 'Team']]
df_2022_opp = df_2022[['Week', 'Opp']]

df_2023_team = df_2023[['Week', 'Team']]
df_2023_opp = df_2023[['Week', 'Opp']]

In [17]:
df_2022_team = pd.merge(df_2022_team, agg_2021, how = 'left', on = 'Team')

for col in df_2022_team.columns:
    if col not in ['Team', 'Week']:
        df_2022_team.rename(columns={col: 'Team_' + col}, inplace=True)
     
     
df_2023_team = pd.merge(df_2023_team, agg_2022, how = 'left', on = 'Team')

for col in df_2023_team.columns:
    if col not in ['Team', 'Week']:
        df_2023_team.rename(columns={col: 'Team_' + col}, inplace=True)   

In [18]:
df_2022_opp = pd.merge(df_2022_opp, agg_2021, how = 'left', left_on = 'Opp', right_on = 'Team')
df_2022_opp = df_2022_opp.drop(columns = 'Team')

for col in df_2022_opp.columns:
    if col not in ['Opp', 'Week']:
        df_2022_opp.rename(columns={col: 'Opp_' + col}, inplace=True)
        
df_2022_opp = df_2022_opp.drop(columns = ['Week'])


df_2023_opp = pd.merge(df_2023_opp, agg_2022, how = 'left', left_on = 'Opp', right_on = 'Team')
df_2023_opp = df_2023_opp.drop(columns = 'Team')

for col in df_2023_opp.columns:
    if col not in ['Opp', 'Week']:
        df_2023_opp.rename(columns={col: 'Opp_' + col}, inplace=True)
        
df_2023_opp = df_2023_opp.drop(columns = ['Week'])

In [19]:
df_2022_agg = pd.concat([df_2022_team, df_2022_opp], axis = 1)
df_2022_agg = pd.concat([df_2022_agg, wl_2022], axis = 1)

df_2023_agg = pd.concat([df_2023_team, df_2023_opp], axis = 1)
df_2023_agg = pd.concat([df_2023_agg, wl_2023], axis = 1)

In [20]:
df_2022_agg['Match'] = df_2022_agg.apply(lambda row: '-'.join(sorted([row['Team'], row['Opp']])), axis=1)
df_2022_agg = df_2022_agg.drop_duplicates(subset=['Match', 'Week'])
df_2022_agg = df_2022_agg.drop(columns=['Match'])

df_2023_agg['Match'] = df_2023_agg.apply(lambda row: '-'.join(sorted([row['Team'], row['Opp']])), axis=1)
df_2023_agg = df_2023_agg.drop_duplicates(subset=['Match', 'Week'])
df_2023_agg = df_2023_agg.drop(columns=['Match'])

df_2022_agg

,Week,Team,Team_Result,Team_OT,Team_H/A,Team_Score_Tm,Team_Score_Opp,Team_Off_1stD,Team_Off_TotYd,Team_Off_PassYd,...,Opp_Off_TO,Opp_Def_1stD,Opp_Def_TotYd,Opp_Def_PassYd,Opp_Def_RushYD,Opp_Def_TO,Opp_Exp_Offense,Opp_Exp_Defense,Opp_Exp_SpecTms,Result
0,1,49ers,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,2.000000,18.500000,271.625000,154.375000,117.250000,1.875000,-2.573750,3.120000,-0.567500,L
1,2,49ers,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,1.625000,21.875000,365.375000,250.375000,115.000000,1.250000,1.265000,-4.305000,1.530000,W
2,3,49ers,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,1.285714,18.142857,317.000000,211.428571,105.571429,1.428571,4.684286,1.810000,-1.164286,L
3,4,49ers,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,2.181818,19.272727,323.090909,235.000000,88.090909,1.727273,2.621818,0.015455,0.578182,W
4,5,49ers,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,2.100000,17.100000,282.400000,178.600000,103.800000,1.500000,-6.894000,4.772000,-0.035000,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,4,saints,0.142857,0.142857,0.571429,16.571429,27.571429,14.571429,283.428571,182.714286,...,1.375000,22.000000,342.875000,218.375000,124.500000,2.000000,2.143750,0.177500,-1.373750,L
446,5,saints,0.142857,0.142857,0.571429,16.571429,27.571429,14.571429,283.428571,182.714286,...,1.625000,21.875000,365.375000,250.375000,115.000000,1.250000,1.265000,-4.305000,1.530000,W
451,10,saints,0.142857,0.142857,0.571429,16.571429,27.571429,14.571429,283.428571,182.714286,...,1.400000,19.800000,365.100000,215.000000,150.100000,1.600000,-1.306000,-2.900000,-0.072000,L
499,8,texans,0.000000,0.000000,0.375000,9.750000,27.375000,12.000000,221.000000,148.750000,...,1.833333,22.500000,361.000000,271.500000,89.500000,1.833333,3.473333,-0.823333,-0.740000,L


['49ers' 'bears' 'bengals' 'bills' 'broncos' 'browns' 'buccaneers'
 'cardinals' 'chargers' 'chiefs' 'colts' 'commanders' 'cowboys' 'dolphins'
 'eagles' 'falcons' 'giants' 'jaguars' 'jets' 'lions' 'packers' 'panthers'
 'patriots' 'raiders' 'rams' 'ravens' 'saints' 'seahawks' 'steelers'
 'texans' 'titans' 'vikings']

In [21]:
team_name_to_number = {
    '49ers': 0,
    'bears': 1,
    'bengals': 2,
    'bills': 3,
    'broncos': 4,
    'browns': 5,
    'buccaneers': 6,
    'cardinals': 7,
    'chargers': 8,
    'chiefs': 9,
    'colts': 10,
    'commanders': 11,
    'cowboys': 12,
    'dolphins': 13,
    'eagles': 14,
    'falcons': 15,
    'giants': 16,
    'jaguars': 17,
    'jets': 18,
    'lions': 19,
    'packers': 20,
    'panthers': 21,
    'patriots': 22,
    'raiders': 23,
    'rams': 24,
    'ravens': 25,
    'saints': 26,
    'seahawks': 27,
    'steelers': 28,
    'texans': 29,
    'titans': 30,
    'vikings': 31
}

df_2022_agg['Team'] = df_2022_agg['Team'].map(team_name_to_number)
df_2022_agg['Opp'] = df_2022_agg['Opp'].map(team_name_to_number)

result_mapping = {'W': 1, 'L': 0}
df_2022_agg['Result'] = df_2022_agg['Result'].map(result_mapping)


df_2023_agg['Team'] = df_2023_agg['Team'].map(team_name_to_number)
df_2023_agg['Opp'] = df_2023_agg['Opp'].map(team_name_to_number)


df_2023_agg['Result'] = df_2023_agg['Result'].map(result_mapping)

In [22]:
df_2022_agg.dropna(subset = ['Result'], inplace=True)
df_2022_agg['Result'] = df_2022_agg['Result'].astype(int)

df_2023_agg.dropna(subset = ['Result'], inplace=True)
df_2023_agg['Result'] = df_2023_agg['Result'].astype(int)

df_2022_agg

,Week,Team,Team_Result,Team_OT,Team_H/A,Team_Score_Tm,Team_Score_Opp,Team_Off_1stD,Team_Off_TotYd,Team_Off_PassYd,...,Opp_Off_TO,Opp_Def_1stD,Opp_Def_TotYd,Opp_Def_PassYd,Opp_Def_RushYD,Opp_Def_TO,Opp_Exp_Offense,Opp_Exp_Defense,Opp_Exp_SpecTms,Result
0,1,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,2.000000,18.500000,271.625000,154.375000,117.250000,1.875000,-2.573750,3.120000,-0.567500,0
1,2,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,1.625000,21.875000,365.375000,250.375000,115.000000,1.250000,1.265000,-4.305000,1.530000,1
2,3,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,1.285714,18.142857,317.000000,211.428571,105.571429,1.428571,4.684286,1.810000,-1.164286,0
3,4,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,2.181818,19.272727,323.090909,235.000000,88.090909,1.727273,2.621818,0.015455,0.578182,1
4,5,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.000000,...,2.100000,17.100000,282.400000,178.600000,103.800000,1.500000,-6.894000,4.772000,-0.035000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,4,26,0.142857,0.142857,0.571429,16.571429,27.571429,14.571429,283.428571,182.714286,...,1.375000,22.000000,342.875000,218.375000,124.500000,2.000000,2.143750,0.177500,-1.373750,0
446,5,26,0.142857,0.142857,0.571429,16.571429,27.571429,14.571429,283.428571,182.714286,...,1.625000,21.875000,365.375000,250.375000,115.000000,1.250000,1.265000,-4.305000,1.530000,1
451,10,26,0.142857,0.142857,0.571429,16.571429,27.571429,14.571429,283.428571,182.714286,...,1.400000,19.800000,365.100000,215.000000,150.100000,1.600000,-1.306000,-2.900000,-0.072000,0
499,8,29,0.000000,0.000000,0.375000,9.750000,27.375000,12.000000,221.000000,148.750000,...,1.833333,22.500000,361.000000,271.500000,89.500000,1.833333,3.473333,-0.823333,-0.740000,0


In [23]:
df_agg_combined = pd.concat([df_2022_agg, df_2023_agg], axis = 0)
df_agg_combined

,Week,Team,Team_Result,Team_OT,Team_H/A,Team_Score_Tm,Team_Score_Opp,Team_Off_1stD,Team_Off_TotYd,Team_Off_PassYd,...,Opp_Off_TO,Opp_Def_1stD,Opp_Def_TotYd,Opp_Def_PassYd,Opp_Def_RushYD,Opp_Def_TO,Opp_Exp_Offense,Opp_Exp_Defense,Opp_Exp_SpecTms,Result
0,1,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.00,...,2.000000,18.500000,271.625000,154.375000,117.250000,1.875000,-2.573750,3.120000,-0.567500,0
1,2,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.00,...,1.625000,21.875000,365.375000,250.375000,115.000000,1.250000,1.265000,-4.305000,1.530000,1
2,3,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.00,...,1.285714,18.142857,317.000000,211.428571,105.571429,1.428571,4.684286,1.810000,-1.164286,0
3,4,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.00,...,2.181818,19.272727,323.090909,235.000000,88.090909,1.727273,2.621818,0.015455,0.578182,1
4,5,0,0.636364,0.090909,0.363636,23.636364,20.636364,18.818182,358.636364,225.00,...,2.100000,17.100000,282.400000,178.600000,103.800000,1.500000,-6.894000,4.772000,-0.035000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,17,27,0.375000,0.125000,0.250000,19.750000,21.500000,17.000000,311.375000,197.25,...,1.400000,19.800000,365.100000,215.000000,150.100000,1.600000,-1.306000,-2.900000,-0.072000,0
479,4,28,0.500000,0.200000,0.500000,20.700000,26.300000,19.500000,318.600000,222.60,...,2.125000,22.250000,360.875000,202.125000,158.750000,1.750000,-12.305000,-6.358750,1.808750,0
483,9,28,0.500000,0.200000,0.500000,20.700000,26.300000,19.500000,318.600000,222.60,...,1.833333,22.500000,361.000000,271.500000,89.500000,1.833333,3.473333,-0.823333,-0.740000,1
506,15,29,0.000000,0.000000,0.375000,9.750000,27.375000,12.000000,221.000000,148.75,...,1.833333,22.500000,361.000000,271.500000,89.500000,1.833333,3.473333,-0.823333,-0.740000,1


In [24]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(df_agg_combined.iloc[:, 1:39], df_agg_combined.iloc[:, 39], test_size = 0.2, random_state = 1)

lr = LogisticRegression(max_iter = 10000)

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.5229357798165137


C:\Users\krizm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
lions_2024 = pd.read_excel('lions_2024.xlsx')
lions_2024

,Team,Opp
0,lions,bears
1,lions,packers
2,lions,vikings
3,lions,buccaneers
4,lions,rams
5,lions,seahawks
6,lions,jaguars
7,lions,titans
8,lions,bills
9,lions,cowboys


In [26]:
lions_2024_team = lions_2024[['Team']]
lions_2024_opp = lions_2024[['Opp']]

lions_2024_team = pd.merge(lions_2024_team, agg_2023, how = 'left', on = 'Team')

for col in lions_2024_team.columns:
    if col not in ['Team']:
        lions_2024_team.rename(columns={col: 'Team_' + col}, inplace=True) 
        
lions_2024_opp = pd.merge(lions_2024_opp, agg_2023, how = 'left', left_on = 'Opp', right_on = 'Team')
lions_2024_opp = lions_2024_opp.drop(columns = 'Team')

for col in lions_2024_opp.columns:
    if col not in ['Opp']:
        lions_2024_opp.rename(columns={col: 'Opp_' + col}, inplace=True)
        

lions_2024_agg = pd.concat([lions_2024_team, lions_2024_opp], axis = 1)

lions_2024_agg['Team'] = lions_2024_agg['Team'].map(team_name_to_number)
lions_2024_agg['Opp'] = lions_2024_agg['Opp'].map(team_name_to_number)

In [27]:
lions_pred = lr.predict(lions_2024_agg)

lions_2024 = pd.concat([lions_2024, pd.DataFrame(lions_pred)], axis = 1)

In [28]:
lions_2024

,Team,Opp,0
0,lions,bears,1
1,lions,packers,0
2,lions,vikings,0
3,lions,buccaneers,0
4,lions,rams,0
5,lions,seahawks,0
6,lions,jaguars,0
7,lions,titans,0
8,lions,bills,0
9,lions,cowboys,0


In [29]:
cowboys_2024 = pd.read_excel('cowboys_2024.xlsx')
cowboys_2024

,Team,Opp
0,cowboys,giants
1,cowboys,eagles
2,cowboys,commanders
3,cowboys,saints
4,cowboys,buccaneers
5,cowboys,lions
6,cowboys,ravels
7,cowboys,bengals
8,cowboys,texans
9,cowboys,falcons


In [30]:
cowboys_2024_team = cowboys_2024[['Team']]
cowboys_2024_opp = cowboys_2024[['Opp']]

In [31]:
cowboys_2024_team = pd.merge(cowboys_2024_team, agg_2023, how = 'left', on='Team')

In [32]:
for col in cowboys_2024_team.columns:
    if col not in ['Team']:
        cowboys_2024_team.rename(columns = {col: 'Team_' + col}, inplace = True)
        
cowboys_2024_opp = pd.merge(cowboys_2024_opp, agg_2023, how = 'left', left_on = 'Opp', right_on = 'Team')
cowboys_2024_opp = cowboys_2024_opp.drop(columns = 'Team') 

for col in cowboys_2024_opp.columns:
    if col not in ['Opp']:
        cowboys_2024_opp.rename(columns={col: 'Opp_' + col}, inplace = True)
        
cowboys_2024_agg = pd.concat([cowboys_2024_team, cowboys_2024_opp], axis = 1)

cowboys_2024_agg['Team'] = cowboys_2024_agg['Team'].map(team_name_to_number)
cowboys_2024_agg['Opp'] = cowboys_2024_agg['Opp'].map(team_name_to_number)

In [33]:
cowboys_2024_agg.fillna(0, inplace=True)
cowboys_pred = lr.predict(cowboys_2024_agg)
cowboys_2024 = pd.concat([cowboys_2024, pd.DataFrame(cowboys_pred)], axis = 1)

In [34]:
cowboys_2024

,Team,Opp,0
0,cowboys,giants,1
1,cowboys,eagles,1
2,cowboys,commanders,1
3,cowboys,saints,1
4,cowboys,buccaneers,1
5,cowboys,lions,1
6,cowboys,ravels,1
7,cowboys,bengals,1
8,cowboys,texans,1
9,cowboys,falcons,1


In [35]:
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(df_agg_combined.iloc[:, 1:39], df_agg_combined.iloc[:, 39], test_size = 0.2, random_state = 1)
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [36]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}, MSE: {mse}, R-squared: {r2}")

MAE: 0.4883272436024729, MSE: 0.32558683050343673, R-squared: -0.3024569472092027
